# Inteligencia de Negocios Geoespacial en Python

> Proyecto en Python que integra inteligencia artificial, geocodificación y análisis geoespacial para mejorar direcciones, obtener coordenadas y asignar clientes automáticamente a oficinas según su ubicación.

## Funcionalidades:
- ✨ Mejora de direcciones con **Google Gemini**
- 📍 Geocodificación con **OpenRouteService**
- 🗺️ Asignación automática a oficinas usando **GeoJSON**
- 📊 Visualización interactiva con **Folium**
- 📁 Salida en Excel con estado de cobertura

## Archivos requeridos:
- `clientes.xlsx`: Datos de entrada con columnas: Departamento, Provincia, Distrito, Dirección
- `cercos.geojson`: Polígonos de cobertura con propiedad `"name"`

### 🧰 Instalar dependencias

Solo se necesita ejecutar una vez.
### Ejecutar solo si no están instaladas
```python
> pip install pandas openrouteservice google-generativeai folium geopandas shapely openpyxl
```
### 🚀Importar Librerias Necesarias

In [41]:
import pandas as pd
import google.generativeai as genai
import openrouteservice
import json
import folium
from shapely.geometry import Point, shape
from openpyxl import load_workbook
import time

### 🔑 Configuración de APIs

In [ ]:
# Gemini API
genai.configure(api_key="KEY GEMINIs")
gemini_model = genai.GenerativeModel("gemini-1.5-flash-latest")

# OpenRouteService API
ors_client = openrouteservice.Client(key="KEY OPEN ROUTE SERVICE")

### 📂 Cargar datos desde Excel

El archivo debe tener las siguientes columnas:
- Departamento
- Provincia
- Distrito
- Direccion

In [ ]:
file_path = "clientes.xlsx"
df = pd.read_excel(file_path)

df.head()

,Departamento,Provincia,Distrito,Direccion
0,Piura,Piura,Morropon,huaylas 110
1,Chiclayo,Chiclayo,Laos,cal Real 220
2,Chiclayo,Chiclayo,Eten,Jr. Amsznas 440
3,Iquitos,Maynas,Belen,Calle Uros 770
4,Tacna,Tacna,Candarave,av Salvvrry 300


### 🧹 Mejorar direcciones con Gemini

Usamos IA para corregir errores tipográficos y estandarizar direcciones.

In [23]:
def mejorar_direccion(direccion):
    prompt = f"""
    Estandariza esta dirección peruana:
    - Usa mayúscula solo donde sea necesario
    - Añade el tipo de vía (Av., Jr., Calle, etc.) si no está claro
    - Devuelve SOLO la dirección mejorada, sin explicaciones adicionales
    
    Dirección: {direccion}
    """
    try:
        response = gemini_model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        print(f"⚠️ Error al mejorar '{direccion}': {e}")
        return direccion

In [24]:
print("🔍 Mejorando direcciones con Gemini...")
df["Direccion"] = df["Direccion"].apply(mejorar_direccion)

🔍 Mejorando direcciones con Gemini...
⚠️ Error al mejorar 'Jr. Huaylas 111': 429 Resource has been exhausted (e.g. check quota).
⚠️ Error al mejorar 'Calle San Martín 222': 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 16
}
]
⚠️ Error al mejorar 'Av. Salaverry 333': 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: 

In [25]:
df.head()

,Departamento,Provincia,Distrito,Direccion
0,Piura,Piura,Morropon,Calle Huaylas 110
1,Chiclayo,Chiclayo,Laos,Calle Real 220
2,Chiclayo,Chiclayo,Eten,Jr. Amazonas 440
3,Iquitos,Maynas,Belen,Calle Uros 770
4,Tacna,Tacna,Candarave,Av. Salverry 300


### 📍 Generar dirección completa

Para mejorar precisión de geocodificador.

In [26]:
df["DireccionCompleta"] = (
    df["Direccion"] + ", " +
    df["Distrito"] + ", " +
    df["Provincia"] + ", " +
    df["Departamento"] + ", Perú"
)

df[["Direccion", "Distrito", "Provincia", "Departamento", "DireccionCompleta"]].head()

,Direccion,Distrito,Provincia,Departamento,DireccionCompleta
0,Calle Huaylas 110,Morropon,Piura,Piura,"Calle Huaylas 110, Morropon, Piura, Piura, Perú"
1,Calle Real 220,Laos,Chiclayo,Chiclayo,"Calle Real 220, Laos, Chiclayo, Chiclayo, Perú"
2,Jr. Amazonas 440,Eten,Chiclayo,Chiclayo,"Jr. Amazonas 440, Eten, Chiclayo, Chiclayo, Perú"
3,Calle Uros 770,Belen,Maynas,Iquitos,"Calle Uros 770, Belen, Maynas, Iquitos, Perú"
4,Av. Salverry 300,Candarave,Tacna,Tacna,"Av. Salverry 300, Candarave, Tacna, Tacna, Perú"


### 🌏 Geocodificación con OpenRouteService

In [27]:
def get_coordinates(address):
    try:
        response = ors_client.pelias_search(address)
        if response and "features" in response and response["features"]:
            coords = response["features"][0]["geometry"]["coordinates"]
            lat, lon = coords[1], coords[0]
            return round(lat, 6), round(lon, 6)
    except Exception as e:
        print(f"❌ Error con '{address}': {e}")
    return "", ""

df[["Latitud", "Longitud"]] = df["DireccionCompleta"].apply(
    lambda x: pd.Series(get_coordinates(x))
)

In [28]:
print("📍 Obteniendo coordenadas con OpenRouteService...")
df[["Latitud", "Longitud"]] = df["DireccionCompleta"].apply(
    lambda x: pd.Series(get_coordinates(x))
)

📍 Obteniendo coordenadas con OpenRouteService...


In [29]:
df.head()

,Departamento,Provincia,Distrito,Direccion,DireccionCompleta,Latitud,Longitud
0,Piura,Piura,Morropon,Calle Huaylas 110,"Calle Huaylas 110, Morropon, Piura, Piura, Perú",-9.518057,-77.531457
1,Chiclayo,Chiclayo,Laos,Calle Real 220,"Calle Real 220, Laos, Chiclayo, Chiclayo, Perú",21.073798,-101.621089
2,Chiclayo,Chiclayo,Eten,Jr. Amazonas 440,"Jr. Amazonas 440, Eten, Chiclayo, Chiclayo, Perú",-6.771557,-79.847250
3,Iquitos,Maynas,Belen,Calle Uros 770,"Calle Uros 770, Belen, Maynas, Iquitos, Perú",-2.690076,-74.000903
4,Tacna,Tacna,Candarave,Av. Salverry 300,"Av. Salverry 300, Candarave, Tacna, Tacna, Perú",-18.028091,-70.272675


### 🧱 Validar si los puntos caen dentro de un polígono (GeoJSON)

Se usa Shapely para validar si un punto está dentro de un polígono.

In [ ]:
def encontrar_oficina(lat, lon):
    if pd.isna(lat) or pd.isna(lon):
        return "Sin cobertura"
    
    point = Point(lon, lat) 

    for feature in geojson_data["features"]:
        polygon = shape(feature["geometry"])
        if polygon.contains(point):
            return feature["properties"]["name"]
    return "Sin cobertura"


df["Cobertura"] = df.apply(lambda row: encontrar_oficina(row["Latitud"], row["Longitud"]), axis=1)

### 📄 Exportar resultados a Excel

In [38]:
output_file = "clientes_con_cobertura.xlsx"
df.to_excel(output_file, index=False, engine="openpyxl")

print(f"✅ Archivo guardado como '{output_file}'")
print("\n📊 Resumen:")
print(df["Cobertura"].value_counts())

✅ Archivo guardado como 'clientes_con_cobertura.xlsx'

📊 Resumen:
Cobertura
Sin cobertura     33
Oficina Norte     16
Oficina Sur       10
Oficina Centro     1
Name: count, dtype: int64


### 🌍 Mapa interactivo con Folium

Muestra los clientes y los cercos definidos en el archivo GeoJSON.

In [39]:
m = folium.Map(location=[-9.19, -75.0152], zoom_start=5)
folium.GeoJson(geojson_data, name="Cercos").add_to(m)

colores_oficinas = {
    "Oficina Norte": "blue",
    "Oficina Centro": "green",
    "Oficina Sur": "red",
    "Sin cobertura": "gray"
}

for idx, row in df.iterrows():
    lat = row["Latitud"]
    lon = row["Longitud"]

    if pd.isna(lat) or pd.isna(lon):
        continue

    cobertura = row["Cobertura"]
    color = colores_oficinas.get(cobertura, "black")
    popup_text = f"{row['Direccion']}<br>{cobertura}"

    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color=color),
        popup=popup_text
    ).add_to(m)

map_file = "mapa_cobertura.html"
m.save(map_file)
print(f"✅ Mapa guardado como '{map_file}'")

✅ Mapa guardado como 'mapa_cobertura.html'


In [40]:
from IPython.display import display
display(m)